# Following the IBM github

https://github.com/IBM/fastfit

# HUGGINGFACE BUGS

- Get all kinds of bugs with accelerate

- https://github.com/huggingface/transformers/issues/34699

Try force wih 4.39.3 seems to work OK 

In [1]:
!pip install transformers==4.37.2
!pip install accelerate==0.28.0
#!pip install -q -U datasets==2.18.0
!pip install fast-fit

In [2]:
from datasets import load_dataset

dataset = load_dataset("data-is-better-together/10k_prompts_ranked")

In [3]:
dataset_tt = dataset["train"].train_test_split(test_size=0.5)

In [4]:
dataset_tt

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 5166
    })
})

# FastFit stuff

In [5]:
from fastfit import FastFitTrainer, sample_dataset

dataset_tt["validation"] = dataset_tt["test"] # i think they expect this naming of tvt keys

print(dataset_tt)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 5166
    })
    validation: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 5166
    })
})


# Vary number of samples for N-shot training

- baseline suggests 5 of each category, i'll do 20

In [6]:
# Down sample the train data for 5-shot training

NUM_SAMPLES_PER_LABEL = 20

dataset_tt["train"] = sample_dataset(dataset_tt["train"], label_column="topic", num_samples_per_label=NUM_SAMPLES_PER_LABEL)

In [7]:
dataset_tt["train"] # should be 10 * NUM_SAMPLES_PER_LABEL since there are 10 different labels in this dataset

Dataset({
    features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
    num_rows: 200
})

In [8]:
# --- ADDED THIS TO AVOID ERROR WHEN DOING EVAL - KAGGLE CRASHES WHEN TRY FULL 5k SAMPLES

SMALL_DATASET = dataset_tt["test"].train_test_split(test_size=0.2)

SMALL_DATASET

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 4132
    })
    test: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 1034
    })
})

## Check text and label fields

In [9]:
dataset_tt["train"][0]

{'prompt': 'Provide a comprehensive list of at least 20 different real-world applications of machine learning, categorized by their specific industry or field of application, and including a brief description of each use case. Additionally, include at least two examples of each category in the form of code snippets or SQL queries that demonstrate how machine learning is being utilized in each context. Finally, discuss the ethical implications of machine learning in each application and provide recommendations for how these issues can be addressed.',
 'quality': [{'user_id': 'cf50b3c3-4c06-4326-ad1b-4b29549214bf',
   'value': '4',
   'status': 'submitted'},
  {'user_id': '33e4b415-177d-4421-b62d-f6839090056e',
   'value': '4',
   'status': 'submitted'},
  {'user_id': '0583afc2-2cd8-43b6-a61b-d73dbf2ad9d9',
   'value': '3',
   'status': 'submitted'}],
 'metadata': '{"source": "evol_instruct", "kind": "synthetic", "evolved_from": null}',
 'avg_rating': 4.666666666666667,
 'num_responses':

- text == prompt
- label == topic

In [10]:
#!pip install accelerate==0.28.0

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"

# I keep getting CUDA out of memory

i don't understand, the base model is 400m or so

on this Kaggle [https://www.kaggle.com/code/aisuko/few-shot-nlp-intent-classification-with-fastfit](https://www.kaggle.com/code/aisuko/few-shot-nlp-intent-classification-with-fastfit) he uses the same defaults as IBM github, except `num_repeats` to 1 but he can do train batch size 32 still ???

I set to 1 to see if it worked and training does start with `1` - I don't understand why I use up 15 Gb GPU with such a small dataset and model though with larger batches!??!

In [13]:
trainer = FastFitTrainer(
    model_name_or_path="sentence-transformers/paraphrase-mpnet-base-v2",
    label_column_name="topic", # <--- "label" goes here
    text_column_name="prompt", # <--- "text" field goes here
    num_train_epochs=1, # DEFAULT WAS 40
    per_device_train_batch_size=1, # 32 gives CUDA error
    per_device_eval_batch_size=1, # 64 gives CUDA error
    max_text_length=512, # default has 128, i checked mpnet max length seems to be 512
    dataloader_drop_last=False,
    num_repeats=4,
    optim="adafactor",
    clf_loss_factor=0.1,
    fp16=True,
    #dataset=dataset_tt,
    train_dataset=dataset_tt["train"],
    validation_dataset=SMALL_DATASET["test"],
    test_dataset=SMALL_DATASET["train"],
)

import time

start_time = time.time()
print("=== STARTING TRAINING OK ===")
model = trainer.train()


end_time = time.time()

print(end_time - start_time)



[WARNING|integration_utils.py:81] 2024-12-15 23:16:08,592 >> Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Flattening the indices:   0%|          | 0/4132 [00:00<?, ? examples/s]

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/200 [00:00<?,…

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/1034 [00:00<?…

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/4132 [00:00<?…

...

/opt/conda/lib/python3.10/site-packages/fastfit/train.py:879: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric(self.data_args.metric_name, experiment_id=uuid.uuid4())
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelera

=== STARTING TRAINING OK ===


[WARNING|modeling_utils.py:1124] 2024-12-15 23:16:26,048 >> Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss


***** train metrics *****
  epoch                    =        1.0
  total_flos               =        0GF
  train_loss               =     2.2747
  train_runtime            = 0:01:07.45
  train_samples            =        200
  train_samples_per_second =      2.965
  train_steps_per_second   =      2.965
67.78096437454224


In [14]:
results = trainer.evaluate()
print("Accuracy: {:.1f}".format(results["eval_accuracy"] * 100))

***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =      0.148
  eval_loss               =     2.1765
  eval_runtime            = 0:02:22.26
  eval_samples            =       1034
  eval_samples_per_second =      7.268
  eval_steps_per_second   =      7.268
Accuracy: 14.8
